In [ ]:
!pip3 install kornia

3) LoFTR  - любой другой, сравнить с готовыми решениями. <br>
Напишите Вывод.

In [33]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
from kornia.geometry import find_homography_dlt
from kornia.feature import LoFTR

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import torch
import time
import cv2


In [36]:
# use LoFTR matcher to get general points from two frames
def process_imgs(matcher, input_dict: dict):
  with torch.inference_mode():
    corrs = matcher(input_dict)

  kps1 = corrs["keypoints0"].reshape(1, -1, 2)
  kps2 = corrs["keypoints1"].reshape(1, -1, 2)

  return kps1, kps2


In [39]:
# functin to find point p1 from first video frame in last frame
def processing_video(output_path: str, p1):
    global INPUT_VIDEO_PATH

    # create empty affine matrix to save affine matrices multiplication results
    # from whole video
    general_matrix_3d = np.array([], dtype=float)
    prev_frame_cuda = None

    cap = cv2.VideoCapture(INPUT_VIDEO_PATH)
    input_width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
    input_height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)

    RESIZED_RESOLUTION = (810, 544)
    # scales for image reduction
    scale_x = input_width / RESIZED_RESOLUTION[0]
    scale_y = input_height / RESIZED_RESOLUTION[1]

    p1[:2] = [p1[0] / scale_x, p1[1] / scale_y]

    # create video writer to make video with point
    fourcc = cv2.VideoWriter.fourcc(*"mp4v")
    video_writer = cv2.VideoWriter(output_path, fourcc, cap.get(cv2.CAP_PROP_FPS),
                                   (int(input_width), int(input_height)))

    # create Local Feature Matching with Transformers
    matcher = LoFTR(pretrained='indoor_new')
    matcher = matcher.eval().cuda()

    while True:
        success, frame = cap.read()

        if not success:
            break

        # convert frame to one-dimential gray image, than resize
        resized_frame_gray = cv2.resize(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY),
                                        RESIZED_RESOLUTION, cv2.INTER_CUBIC)

        # reshape image to get shape (batch x channels x H x W)
        resized_frame_gray = resized_frame_gray.reshape(1, 1,
                                                        RESIZED_RESOLUTION[1],
                                                        RESIZED_RESOLUTION[0])
        frame_cuda = torch.from_numpy(resized_frame_gray).cuda() / 255.

        if prev_frame_cuda is not None:
            input_dict = {'image0': prev_frame_cuda, 'image1': frame_cuda}
            kps1, kps2 = process_imgs(matcher, input_dict)

            # get affine 3D-matrix between two alternate frames
            affine_matrix_3d = find_homography_dlt(kps1, kps2).cpu().numpy()[0]
            general_matrix_3d = (affine_matrix_3d if general_matrix_3d.size == 0
                                 else np.dot(affine_matrix_3d, general_matrix_3d))

            # calculate p1 coordinates in current frame, for visualization only
            p2 = np.dot(general_matrix_3d, p1)
            frame = cv2.circle(frame, (int(p2[0] * scale_x),
                                       int(p2[1] * scale_y)),
                               10, (0, 0, 255), -1)
        else:
            frame = cv2.circle(frame, (int(p1[0] * scale_x),
                                       int(p1[1] * scale_y)),
                               10, (0, 0, 255), -1)
        video_writer.write(frame)

        # save current resized gray frame as previous to calculate affine matrix in next step
        prev_frame_cuda = frame_cuda

    cap.release()
    video_writer.release()

    # rescale p2 point
    p2[:2] = [p2[0] * scale_x, p2[1] * scale_y]

    return p2


In [40]:
INPUT_VIDEO_PATH = '/content/source_video.mp4'
OUTPUT_VIDEO_PATH = '/content/drive/MyDrive/video_with_point'

df_columns = ['Descriptor method', 'Time,s', 'Error,px']
df_file_path = '/content/drive/MyDrive/LoFRT_video_analyze.xlsx'
analyze_df = pd.DataFrame(columns=df_columns)

p1 = [1430, 470, 1]
p2_target = [1020, 400, 1]

output_path = OUTPUT_VIDEO_PATH + '_' + 'LoFTR.mp4'

start_t = time.time()
p2 = processing_video(output_path, p1.copy())
delta_t = round(time.time() - start_t, 4)

points_dist = np.sqrt((p2[0] - p1[0]) ** 2 + (p2[1] - p1[1]) ** 2)
error_dist = np.sqrt((p2[0] - p2_target[0]) ** 2 + (p2[1] - p2_target[1]) ** 2)

df_row = ['LoFTR', delta_t, round(error_dist, 4)]

if analyze_df.empty:
    analyze_df = pd.DataFrame([df_row], columns=df_columns)
else:
    analyze_df = analyze_df._append(dict(zip(df_columns, df_row)),
                                    ignore_index=True)
analyze_df.to_excel(df_file_path, index=False)


In [41]:
print(analyze_df)


  Descriptor method    Time,s   Error,px
0             LoFTR  853.0437  8166.7404
